In [29]:
import os
# Check if you're on Google drive or on your own machine.
# Get path to your data.
if ('google' in str(get_ipython())):
    from google.colab import drive
    drive.mount('ME')
    #predir='/content/ME/My Drive/'
    predir='ME/My Drive/'
else:
    predir='Desktop/college/'
    # Please specify your own local directory containing the data and labels
   
import torch
import numpy as np
import copy
# Torch functions 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time
import numpy as np
import collections
import pickle
import argparse
from random import shuffle

from torch.autograd import Variable
import matplotlib.pylab  as plt
# datadir=predir+'LSDA_data/NLP/'
datadir=predir+'ideepwise/NLP/task1/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Utility to track progress of a routine.
#from tqdm import tqdm
from tqdm.notebook import trange, tqdm

# Folder with course data
print("TORCH.cuda", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("datadir:", datadir)
print("get_ipython:", get_ipython())

!python --version
os.getcwd()

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D



Drive already mounted at ME; to attempt to forcibly remount, call drive.mount("ME", force_remount=True).
TORCH.cuda False
datadir: ME/My Drive/ideepwise/NLP/task1/
get_ipython: <google.colab._shell.Shell object at 0x7ff6992e9210>
Python 3.7.13


In [30]:
# workflow that separates input data and their labels
def divide_data_and_label(input_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        file = f.read()
    # print(data)
    data = []
    temp = []
    labels = []
    temp_file = []
    for char in file:
        if char == '\n':
            temp_file.append(temp)
            temp = []    
        elif char == '\s' or char == '\t':
            temp_file.append(temp)
            temp = []
        else:
            temp.append(char)
    for i in range(len(temp_file)):
        if i % 2 == 0:
            data.append(temp_file[i])
        if i % 2 == 1:
            labels.append(temp_file[i])
    
    for i in range(len(data)):
        data[i] = "".join(data[i])
        labels[i] = "".join(labels[i])
    return (data, labels)

In [31]:
data_train, labels_train = divide_data_and_label(os.path.join(datadir, 'train.txt'))
data_dev, labels_dev = divide_data_and_label(os.path.join(datadir, 'dev.txt'))
print(data_train)
print(labels_train)

['新建联系人张三电话号码一二三四五', '黄狼。', '把张玉娟的手机号码发送给吴伟', '潘靖的电话是多少', '新建联系人买当唠4008 517517', '新建联系人天天', '新建联系人王满秀幺三七 六四零七零二九五', '新建联系人王义勇幺三八六零零幺九四九零', '新建联系人孙晓飞幺三八三八四四三八九五', '查找张伟军', '把平水相逢的手机号码发给8000', '把小新的号码发给徐云蔚', '新建联系人振华奎号码是幺三五零八九四九三幺二', '把徐程的电话发给吴芳', '把哥哥的电话发给殷龙', '把李文鼎的号码发给谢服全', '把侨天慧的号码发给副卡', '查看婷婷的号码', '新建联系人张雨号码是五六七', '周慧的电话。', '把唐玉明的手机号码发给李康', '查看王老师的号码', '新建联系人18622625490', '把李志强的号码发给贾洪鉴', '新建联系人姓名张三号码一二三四五六七八', '查寻标兵的号码', '把许琦彪的号码发给徐鹏', '添加一条通讯录姓名葛勇手机号码189571642496', '把王世怀的号码发给乔丽君', '把张新涛电话发给常胜宾', '到太湖怎么走', '去常州武进路线', '湖里公园在哪里', '带我去丹阳市眼镜市场', '从南京到亳州怎么走', '上海到苏州', '美国大使馆在哪', '华晶小学在哪里', '去科大讯飞怎么走', '导航到佛山大良汽车站', '松江钢材城在哪里', '我想到科大讯飞怎么走', '去湘江', '合肥百货大楼在哪里', '往火车北站怎么走', '辽宁大连在哪里', '到步行街不走高架的路线是哪条', '到鼓楼怎么走', '合肥的大润发在哪', '怎么去泉城广场', '到合肥市逍遥津公园怎么走', '沿途有没有加油站', '到环都大酒店怎么走', '到公园怎么走', '我要去姜堰', '六安市飞云卫生院在哪里', '科大讯飞在哪里呀', '从这里到市中心怎么走', '查询北京到上海的路线', '讯飞语点我现在在哪里', '春熙路怎么走', '怎么去龙阳路', '到郑州火车站', '宁波机场在哪里', '从二手车市场到八一广场怎么走', '帮我查一下合肥市三孝口在哪里', '我所在地理位置', '帮我搜一下科大讯飞在哪', '查询沿途的KTV', '郑州火车站怎么走', '鹰潭

In [32]:
# preprocess the data using simple vocabulary 

def preprocess(data, vocab_to_idx=None):
    counter = collections.Counter()
    for sentence in data:
        counter.update(sentence)
    print(counter)
    count_pairs = sorted(counter.items(), key=lambda x: -x[1])
    # print("The counter_pairs are", count_pairs)
    chars, _ = zip(*count_pairs)
    # print("self.chars are:", chars)
    vocab_size = len(chars)
    # print("vocab_size are", vocab_size)
    if vocab_to_idx is None:
        vocab_to_idx = dict(zip(chars, range(len(chars))))
        # print("vocab_to_idx is:", vocab_to_idx)
        idx_to_vocab = dict(zip(vocab_to_idx.values(), vocab_to_idx.keys()))
        # print("idx_to_vocab is:", idx_to_vocab)
    # appending new words in the vocabulary
    for i in range(len(chars)):
        n = len(vocab_to_idx)
        if vocab_to_idx.get(chars[i]) is None:
            vocab_to_idx[chars[i]] = n
            n +=1

    tensor = np.array([list(map(vocab_to_idx.get, data[i])) for i in range(len(data))])
    # print(tensor)
    return vocab_to_idx, tensor

def preprocess_labels(labels, label_dict=None):
    label_set = set(labels) 
    num = len(label_set)
    if label_dict is None:
        i = 0
        label_dict = {}
        for label in label_set:
            label_dict[label] = i
            i += 1
    ret = labels[:]
    for i in range(len(labels)):
        ret[i] = label_dict[labels[i]]
    print(label_dict)
    return ret, num, label_dict

In [55]:
vocab_to_idx, tensor_train = preprocess(data_train)
_, tensor_dev = preprocess(data_dev, vocab_to_idx)
print(tensor_train)

Counter({'的': 611, '么': 595, '我': 411, '。': 341, '怎': 330, '你': 318, '一': 251, '？': 251, '什': 236, '天': 222, '是': 209, '做': 208, '电': 204, '到': 175, '给': 170, '吗': 143, '下': 139, '查': 136, '有': 132, '打': 125, '看': 122, '新': 118, '车': 115, '说': 114, '在': 107, '上': 103, '首': 102, '想': 96, '发': 94, '不': 94, '开': 94, '，': 94, '视': 92, '影': 91, '票': 90, '个': 88, '好': 85, '哪': 84, '大': 84, '州': 82, '了': 82, '明': 81, '要': 80, '帮': 80, '播': 78, '火': 76, '中': 76, '听': 75, '台': 75, '来': 75, '去': 71, '信': 71, '话': 70, '闻': 69, '股': 69, '南': 67, '气': 67, '道': 64, '啊': 64, '人': 62, '1': 62, '小': 62, '海': 59, '这': 59, '今': 59, '法': 59, '最': 57, '英': 56, '放': 55, '三': 54, '搜': 54, '诗': 54, ' ': 53, '京': 53, '肉': 51, '时': 50, '谁': 49, '短': 49, '零': 48, '广': 48, '网': 48, '飞': 47, '航': 47, '0': 46, '语': 46, '鱼': 46, '节': 46, '机': 45, '国': 45, '班': 45, '号': 44, '方': 44, '吃': 44, '北': 43, '频': 42, '索': 42, '呢': 42, '歌': 42, '二': 41, '乐': 41, '目': 41, '八': 40, '里': 40, '红': 39, '文': 38, '现': 38, '五': 37, '

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [34]:
class_train, num, label_dict = preprocess_labels(labels_train)
class_dev, _, _  = preprocess_labels(labels_dev, label_dict=label_dict)
# print(class_train)
# print(labels_train)


{'email': 0, 'poetry': 1, 'translation': 2, 'weather': 3, 'stock': 4, 'message': 5, 'train': 6, 'match': 7, 'datetime': 8, 'website': 9, 'music': 10, 'app': 11, 'cookbook': 12, 'chat': 13, 'lottery': 14, 'riddle': 15, 'health': 16, 'bus': 17, 'video': 18, 'flight': 19, 'radio': 20, 'map': 21, 'epg': 22, 'cinemas': 23, 'news': 24, 'schedule': 25, 'novel': 26, 'telephone': 27, 'tvchannel': 28, 'calc': 29, 'contacts': 30}
{'email': 0, 'poetry': 1, 'translation': 2, 'weather': 3, 'stock': 4, 'message': 5, 'train': 6, 'match': 7, 'datetime': 8, 'website': 9, 'music': 10, 'app': 11, 'cookbook': 12, 'chat': 13, 'lottery': 14, 'riddle': 15, 'health': 16, 'bus': 17, 'video': 18, 'flight': 19, 'radio': 20, 'map': 21, 'epg': 22, 'cinemas': 23, 'news': 24, 'schedule': 25, 'novel': 26, 'telephone': 27, 'tvchannel': 28, 'calc': 29, 'contacts': 30}


In [35]:
import keras
# print(class_train)
# print(class_dev)
y_train = keras.utils.np_utils.to_categorical(class_train, num_classes=31)
y_dev = keras.utils.np_utils.to_categorical(class_dev, num_classes=31)
# a = keras.constant(a,shape=[31,31])
# print(y_train)
# print(np.argmax(y_train, axis=1))
# print(np.argmax(y_dev, axis=1))


In [36]:
np.random.seed(7)
sequence_train = {len(data_train[i]):i for i in range(len(data_train))}
sorted_sequnece_train = sorted(sequence_train.items(), key=lambda x: -x[0])
sequence_dev = {len(data_dev[i]):i for i in range(len(data_dev))}
sorted_sequnece_dev = sorted(sequence_dev.items(), key=lambda x: -x[0])
print(sorted_sequnece_train)
print(sorted_sequnece_dev)
max_length_train = max([len(data_train[i])] for i in range(len(data_train)))
max_length_dev = max([len(data_dev[i])] for i in range(len(data_dev)))
max_length = max(max_length_train, max_length_dev)[0]
print(max_length)
print(data_dev[sorted_sequnece_dev[0][1]])
# print(max_length)

[(48, 447), (37, 438), (32, 1086), (30, 610), (27, 881), (26, 118), (25, 237), (23, 1298), (22, 2229), (21, 2087), (20, 1690), (19, 1532), (18, 2264), (17, 2294), (16, 2284), (15, 2273), (14, 2210), (13, 2272), (12, 2276), (11, 2281), (10, 2296), (9, 2293), (8, 2289), (7, 2297), (6, 2298), (5, 2261), (4, 2259), (3, 2242), (2, 2137)]
[(65, 117), (24, 304), (23, 237), (22, 670), (21, 661), (20, 376), (19, 405), (18, 762), (17, 737), (16, 718), (15, 702), (14, 675), (13, 690), (12, 767), (11, 763), (10, 765), (9, 769), (8, 766), (7, 754), (6, 764), (5, 761), (4, 753), (3, 606), (2, 638)]
65
就不告诉你又能咋样我你再跟我那么叫我起床！跟我说句话！估计你给我发个图或者跟你给我发个表情我才能告诉你！要不然你就得跟我换个秘密！


In [37]:
def pad_embedding(input, max_length):
    input = sequence.pad_sequences(input, maxlen=max_length)
    return input

In [38]:
print(tensor_train)

tensor_train = pad_embedding(tensor_train, max_length)
tensor_dev = pad_embedding(tensor_dev, max_length)
print(np.array(tensor_train).shape)
print(np.array(tensor_dev).shape)


[list([21, 220, 275, 314, 59, 131, 69, 12, 52, 90, 143, 6, 98, 69, 112, 106])
 list([361, 607, 3])
 list([209, 131, 477, 871, 0, 244, 87, 90, 143, 28, 527, 14, 528, 529])
 ... list([670, 299, 61, 23, 869, 187, 191, 1122, 1123, 870])
 list([2, 42, 20, 391, 754, 61, 23])
 list([869, 187, 191, 1122, 1123, 870])]
(2299, 65)
(770, 65)


In [39]:
class_train = np.array(class_train)
class_dev = np.array(class_dev)
print(class_train.shape)
print(y_train.shape)

(2299,)
(2299, 31)


In [50]:
from sklearn.metrics import precision_recall_fscore_support


def run_model (max_length, num, tensor_train, y_train, tensor_dev, y_dev):
    embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(2000, embedding_vecor_length, input_length=max_length))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(128))
    # model.add(Bidirectional(LSTM(128, return_sequences=True))
      # model.add(Bidirectional(LSTM(128)))
    # model.add(Dropout(0.2))
    
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.2))
    model.add(Dense(num, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    print(tensor_train.shape)

    output = model.fit(tensor_train, y_train, epochs=50, batch_size=64)
    # Final evaluation of the model
    scores = model.evaluate(tensor_dev, y_dev, verbose=1)
    print("Accuracy: %.2f%%" % (scores[1]*100))

    # macro F1:
    train_predict = np.argmax(model.predict(tensor_train),axis=1)


    dev_predict = np.argmax(model.predict(tensor_dev),axis=1)
    precision_recall_fscore_support(class_train, train_predict, average='macro')

    # macro F1 score for dev

    precision_recall_fscore_support(class_dev, dev_predict, average='macro')
    print("all done")



In [51]:
run_model(max_length, num, tensor_train, y_train, tensor_dev, y_dev)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 65, 32)            64000     
                                                                 
 conv1d_1 (Conv1D)           (None, 65, 32)            3104      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 32, 32)           0         
 1D)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 128)               82432     
                                                                 
 dense_1 (Dense)             (None, 31)                3999      
                                                                 
Total params: 153,535
Trainable params: 153,535
Non-trainable params: 0
________________________________________________

总结：于dev set准确率达到82.21，同时取得 (准确率 = 0.793112136832583, 召回率 = 0.7797599642922365, F1值 = 0.7745372927915349)的评分 

# *Retrain the model using w2*

In [41]:
import logging
from gensim import utils
import gensim.models

# model = gensim.models.Word2Vec(sentence=)
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [42]:
!pip install jieba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:
from keras import datasets
# testing:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models.word2vec import LineSentence
import gensim.models
import jieba

def split_chinese_sentence(data, cut_all=False):
    data_copy = data[:]
    for i in range(len(data)):
        data_copy[i] = list(jieba.cut(data[i], cut_all))
    return data_copy


In [97]:
train_sentences = split_chinese_sentence(data_train, False)
dev_sentences = split_chinese_sentence(data_dev, False)
print(train_sentences)
sentences = train_sentences + dev_sentences
# print(sentences)
w2v_model = gensim.models.Word2Vec(sentences)
w2v_weights = w2v_model.wv.vectors

print(w2v_model.wv.index2word)

# print(w2v_model.wv.get("新建"))
# print(w2v_model.wv.get_vector("新建"))
print(w2v_model.wv["新建"])
print(w2v_model.wv["联系人"])
print(w2v_model.wv["电话号码"])
# train_sentences = split_chinese_sentence(data_train, False)
# dev_sentences = split_chinese_sentence(data_train, False)


w2v_tensor_train = np.array([list(map(w2v_model.wv.get_vector, train_sentences[i])) for i in range(len(train_sentences))])
w2v_tensor_dev = np.array([list(map(w2v_model.wv.get_vector, dev_sentences[i])) for i in range(len(dev_sentences))])
# tensor = np.array([list(map(vocab_to_idx.get, data[i])) for i in range(len(data))

# w2v_tensor_train = np.array([list(map(w2v))])

[['新建', '联系人', '张三', '电话号码', '一二三四五'], ['黄狼', '。'], ['把', '张玉娟', '的', '手机号码', '发送给', '吴伟'], ['潘靖', '的', '电话', '是', '多少'], ['新建', '联系人', '买', '当', '唠', '4008', ' ', '517517'], ['新建', '联系人', '天天'], ['新建', '联系人', '王满秀', '幺', '三七', ' ', '六四', '零七', '零二', '九五'], ['新建', '联系人', '王', '义勇', '幺', '三八', '六', '零零', '幺', '九四', '九零'], ['新建', '联系人', '孙晓飞', '幺', '三八', '三八', '四四', '三八', '九五'], ['查找', '张伟军'], ['把', '平水', '相逢', '的', '手机号码', '发给', '8000'], ['把', '小', '新', '的', '号码', '发给', '徐云蔚'], ['新建', '联系人', '振华', '奎', '号码', '是', '幺', '三五', '零', '八九', '四九', '三', '幺', '二'], ['把', '徐程', '的', '电话', '发给', '吴芳'], ['把', '哥哥', '的', '电话', '发给', '殷龙'], ['把', '李文鼎', '的', '号码', '发给', '谢服', '全'], ['把', '侨', '天慧', '的', '号码', '发给', '副', '卡'], ['查看', '婷婷', '的', '号码'], ['新建', '联系人', '张雨', '号码', '是', '五', '六七'], ['周慧', '的', '电话', '。'], ['把', '唐玉明', '的', '手机号码', '发给', '李康'], ['查看', '王老师', '的', '号码'], ['新建', '联系人', '18622625490'], ['把', '李', '志强', '的', '号码', '发给', '贾洪鉴'], ['新建', '联系人', '姓名', '张三', '号码', '一二三四五', '六', '七八'

KeyError: ignored

In [ ]:

run_model (317, num, w2v_weights_train, y_train, w2v_weights_dev, y_dev)